install yfinance


In [5]:
!pip install yfinance

import the modules


In [17]:
import yfinance as yf
import pandas as pd

# Adjust the start and end dates
start_date = "2024-01-01"  # Example: Within the last 60 days
end_date = "2024-03-04"    # Example: Today's date

# Download data
dataF = yf.download("EURUSD=X", start=start_date, end=end_date)

# Print the data
#print(dataF)

dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-01,1.105583,1.105339,1.103997,1.105583,1.105583,0
2024-01-02,1.103875,1.104606,1.094200,1.103875,1.103875,0
2024-01-03,1.094176,1.096700,1.089871,1.094176,1.094176,0
2024-01-04,1.092777,1.097021,1.091703,1.092777,1.092777,0
2024-01-05,1.094739,1.099638,1.087985,1.094739,1.094739,0
2024-01-08,1.094224,1.097815,1.092454,1.094224,1.094224,0
2024-01-09,1.095170,1.096732,1.091262,1.095170,1.095170,0
2024-01-10,1.093243,1.096972,1.092335,1.093243,1.093243,0
2024-01-11,1.097514,1.099143,1.093147,1.097514,1.097514,0
